In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_openml

from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
#mycmap = ListedColormap(["blue","red","gold","green","purple","orange","pink","brown","gray"])
mycmap = ListedColormap(["blue","red", "gold"])
import plotly.graph_objects as go


In [ ]:
E = np.zeros(h_generated.shape[0])
for i in range(len(E)):
    v = v_generated[i,:]
    h = h_generated[i,:]
    E[i] = -np.dot(v, a) -np.dot(h, b) - np.dot(v, np.dot(h, w))


In [ ]:
grid = np.linspace(-5, 5, 30)
xx, yy, zz = np.meshgrid(grid, grid, grid)

# generate h and v for each point in the grid
# apply inverse PCA and inverse scaler to find h
h_generated = scaler.inverse_transform(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel(), zz.ravel()]))
v_generated = hidden_to_visible(h_generated, model)

# Contrasting divergence using gibbs sampling
for _ in range(3):
    v_generated = model.gibbs(v_generated)


In [ ]:
# Perform PCA (kernel='linear' for the usual PCA)
pca = KernelPCA(n_components=3, kernel='linear', fit_inverse_transform=True)
X_pca = pca.fit_transform(scaled_data)

#print("Explained variance from the first 3 components:", np.sum(pca.explained_variance_ratio_))

fig = go.Figure(data=[go.Scatter3d(x=X_pca[:,0], y=X_pca[:,1], z=X_pca[:,2], mode='markers',
				marker=dict(size=5, color=label, colorscale=mycmap.colors, opacity=0.8))])
fig.update_layout(width=800, height=800, margin=dict(l=0,r=0,b=0,t=0,pad=0))
fig.show()


In [ ]:
# Perform PCA (kernel='linear' for the usual PCA)

KERNEL = ['linear', 'poly', 'rbf', 'sigmoid']
fig, axs = plt.subplots(1, 4, figsize=(14, 4))

for i in range(len(KERNEL)):
    pca = KernelPCA(n_components=2, kernel=KERNEL[i], fit_inverse_transform=True, gamma=0.05)
    X_pca = pca.fit_transform(scaled_data)

    axs[i].scatter(X_pca[:, 0], X_pca[:, 1], c=label, cmap=mycmap, s=10)
    axs[i].set_title(f"KernelPCA - {KERNEL[i]}")
    axs[i].set_xticks([])
    axs[i].set_yticks([])
plt.tight_layout()
plt.colorbar(axs[0].collections[0], ax=axs, orientation='vertical', fraction=0.02, pad=0.04)
plt.show()


In [ ]:
# Train RBM
from sklearn import linear_model
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline

"https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html"
model = BernoulliRBM(n_components=24, learning_rate=0.1, n_iter=200, verbose=0, random_state=0)
RBM_fit = model.fit(data, label)

# Extract the weights and biases
w = model.components_
a = model.intercept_visible_
b = model.intercept_hidden_

# Extract the hidden layer
h = model.transform(data)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(h)


In [ ]:
def sigmoid(x):
    """Compute the sigmoid activation function."""
    x = np.clip(x, -500, 500)  # Avoid overflow
    return 1 / (1 + np.exp(-x))

def hidden_to_visible(hidden, model):
    w = model.components_
    a = model.intercept_visible_
    prob_visible = sigmoid(np.dot(hidden, w) + a)
    v = (prob_visible > np.random.rand(*prob_visible.shape)).astype(int)
    return v

def visible_to_hidden(visible, model):
    w = model.components_
    b = model.intercept_hidden_
    prob_hidden = sigmoid(np.dot(w, visible) + b)
    h = (prob_hidden > np.random.rand(*prob_hidden.shape)).astype(int)
    return h


In [ ]:
pixels, label = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
label = label.astype(int)
pixels = np.where(pixels / np.max(pixels) > 0.5, 1, 0).astype(int)

dataframe = { "label": label, "pixels": [px for px in pixels] }
df = pd.DataFrame(dataframe)

N = 1000
dataset = df[df["label"] <= 2] # select only 0, 1, 2 values
valSet = dataset[N:N+400]
dataset = dataset[0:N]
data = dataset["pixels"].to_list()
label = dataset["label"].to_list()


In [ ]:
fig = go.Figure(data=go.Volume(
    x=xx.flatten(), y=yy.flatten(), z=zz.flatten(),
    value=E,
    #isomin=0.2,
    #isomax=0.7,
    opacity=0.05,
    surface_count=30,
    colorscale="Magma",
    reversescale=True,
    colorbar={"title": "Energy"}
    ))

fig.add_trace(go.Scatter3d(
    x=X_pca[:,0],
    y=X_pca[:,1],
    z=X_pca[:,2],
    mode="markers",
    marker=dict(
        size=3,
        color=label,
        colorscale=mycmap.colors,  # Use labels to color points
    ),
    name="Hidden Points"
))


fig.update_layout(width=800, height=800, title="Energy Landscape of RBM",
                  scene=dict(
                      xaxis_title="PCA0",
                      yaxis_title="PCA1",
                      zaxis_title="PCA2"
                  ))
fig.show()
